# Basic Agent

## Deps

In [1]:
!pip install --upgrade pip
!pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api -U -q

## OpenAI API Key

In [2]:
api_key = "sk-proj-..."

## Config

### Libs

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import AgentWorkflow, ToolCallResult, AgentStream

### LLM

In [4]:
llm = OpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    api_key=api_key
)

## Agent

In [5]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> float:
    """Divide two numbers"""
    return a / b


agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[add, subtract, multiply, divide],
    llm=llm,
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools.",
)

## Run/Thought

### Without Context

In [6]:
handler = None
handler = agent.run("What is ((2 + 2) * 2 / 100) - 25?")

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
        print("")
    elif isinstance(ev, AgentStream):
        print(ev.delta, end="", flush=True)

resp = await handler

Called tool:  add {'a': 2, 'b': 2} => 4

Called tool:  multiply {'a': 4, 'b': 2} => 8

Called tool:  divide {'a': 8, 'b': 100} => 0.08

Called tool:  subtract {'a': 25, 'b': 25} => 0

Called tool:  subtract {'a': 0.08, 'b': 0} => 0.08

The result of the expression ((2 + 2) * 2 / 100) - 25 is 0.08.

### With Context

In [8]:
from llama_index.core.workflow import Context

ctx = Context(agent)

response = None
response = await agent.run("What is ((2 + 2) * 2 / 100) - 25?", ctx=ctx) # -24.92
response = await agent.run("Add 25 to the result", ctx=ctx) # 0.08
response = await agent.run("Result * 100", ctx=ctx) # 8
response = await agent.run("Divide then by 2", ctx=ctx) # 8/2 = 4

response.response.blocks[0].text

'Dividing the result by 2 gives 4.0.'